## Helper function to load and save images in the appropriate format

In [1]:
import numpy as np
from PIL import Image
def loadImgToArray(url,info=False):
    """
    take the url and return the image (PIL format) as a numpy Array
    """
    with Image.open(url) as image:
    #image = Image.open(url)
        imgArray = np.asarray(image)
    if info:
        print(f'From: {type(image)}, Size: {image.size}, Mode: {image.mode}\nTo: {type(imgArray)}, shape: {imgArray.shape}')
    
    return imgArray

In [2]:
def saveArrayToImg(url,npArray,info=False):
    """
    Takes a numpy array and converts it 
    """
    image = Image.fromarray(npArray)
    if info:
        print(f'From: {type(npArray)}, shape: {npArray.shape}\nTo: {type(image)}, Size: {image.size}, Mode: {image.mode}')
        
    image.save(url)
    image.close()

In [3]:
# test
# mainUrl = '/home/henri/Desktop/roboflow/quickTest/'
# testUrl = '/home/henri/Desktop/roboflow/quickTest/'

# imgArray = loadImgToArray(mainUrl+'2.jpg', info=True)
# saveArrayToImg(testUrl+'.jpg', imgArray,info=True)

## Helper functions

In [4]:
from io import StringIO 
def loadingBBox(fileurl,imageArray,info=True):
    """
    Assuming the data is in this format (yolo)
    <object-class> <x> <y> <width> <height> 
    [1.      , 0.250126, 0.29366 , 0.471062, 0.410735]
    the fucntion will return the following format
    <object-class> <x1> <y1> <x2> <y2>

    The following code allowed to push through
    https://github.com/dnissimi/imgaug-yolov3/blob/master/imgaug-yolov3.py

    *rant warning*
    legit the reason why it is so complex is that the yolo format values are kept between 0-1 and that they are multiplied by the file's shape
    so they have to be 'hydrated' by multiplying each values by their corresponding image height or width.
    i.e. if a box width and height are both .5 that mean the box essentially takes half of the screen area.
    The only thing I don't quite understand in the calculations is why Wbox/hbox have to be divided by 2. Clue yolo x,y indicates the center of the box so you have to push the coordinate to the left by half (since the value is in the center)

    """
    # width/height of the orginal image
    width = imageArray.shape[1] #width or x axis
    height = imageArray.shape[0] #height of y axis
    boxData = np.loadtxt(fileurl)
    
    x1 = boxData[:,1:2:] * width - (boxData[:,3:4:] * width / 2)
    y1 = boxData[:,2:3:] * height - (boxData[:,4:5:] * height / 2)
    x2 = boxData[:,3:4:] * width + x1
    y2 = boxData[:,4:5:] * height + y1
    scaledtransformed = np.column_stack([boxData[:,0:1:],x1,y1,x2,y2])
    if info:
        print(f'\nimage width: {width} and height {height}')
        print(f'before transform <object-class> <x> <y> <width> <height>:\n{boxData}\n after transform <object-class> <x1> <y1> <x2> <y2>:\n{scaledtransformed}')

    return scaledtransformed

In [5]:
# test (loading boxes)
mainUrl = '/home/henri/Desktop/roboflow/'
testUrl = '/home/henri/Desktop/roboflow/quickTest/'
# npArray = loadImgToArray(testUrl+'60.jpg', info=True)
# boxData = loadingBBox(testUrl+'60.txt', imgArray, info=True)

In [6]:
def saveBBox(fileurl,boxData,imageArray,info=True):
    """
    Assuming the data is in this format (yolo)
    <object-class> <x1> <y1> <x2> <y2> 
    the fucntion will return the following format
    <object-class> <x> <y> <width> <height>

    Doesn't take into account boxes that fell outside the image size post transformation
    """
    # width/height of the transfomer image
    width = imageArray.shape[1] #width or x axis
    height = imageArray.shape[0] #height of y axis

    x_pos = boxData[:,1:2:] / width - (( boxData[:,1:2:] - boxData[:,3:4:]) / width / 2)
    y_pos = boxData[:,2:3:] / height - ((boxData[:,2:3:] - boxData[:,4:5:]) / height /2)
    # the original repo had an error there
    x_size = (boxData[:,3:4:] - boxData[:,1:2:]) / width 
    y_size = (boxData[:,4:5:] - boxData[:,2:3:]) / height
    scaledtransformed  = np.column_stack([boxData[:,0:1:],x_pos,y_pos,x_size,y_size])

    # normally we 
    if info:
        print(f'\nimage width: {width} and height {height}')
        print(f'before transform\n<object-class> <x1> <y1> <x2> <y2>:\n{boxData} after transform <object-class> <x> <y> <width> <height>:\n{scaledtransformed}')
    
    np.savetxt(fileurl, scaledtransformed,fmt='%f', delimiter=' ')

In [7]:
def showImg(imgArray):
    """
    takes a numpy array and display as a picture
    """  
    imgplot = plt.imshow(imgArray)
    plt.show()

In [8]:
def bboxNPPacker(bbox):
    packer = np.empty([len(bbox.bounding_boxes), 5])
    for idx in range(len(bbox.bounding_boxes)):
        bboxIdx = bbox.bounding_boxes[idx]
        packer[idx] = [bboxIdx.label,bboxIdx.x1,bboxIdx.y1,bboxIdx.x2,bboxIdx.y2]
    return packer

In [9]:
# saveBBox(testUrl+'136.txt', boxData, imgArray, info=True)

In [10]:
def fileNameAugGen(i,fileName):
    oldJpgName = fileName[0].split('.')
    oldTxtName = fileName[1].split('.')
    newName = [oldJpgName[0] +f'aug{i}.' + oldJpgName[1],oldTxtName[0] +f'aug{i}.' + oldTxtName[1]]
    return newName

In [11]:
def augmentedCopiesSingleImage(imgArray,bbox,amount,info=False):
    results = dict()
    for i in range(0,amount):
        results[i] = seq(image=imgArray, bounding_boxes=bbox)
    return results

In [12]:
from os import listdir
from os.path import isfile, join
mainUrl = '/home/henri/Desktop/roboflow/'

def directoryImgTxtload(mainUrl):
    """
    function that takes the directory link and package jpg and txt files into a list.
    Check for mismatch
    help from: https://stackoverflow.com/questions/21752610/iterate-every-2-elements-from-list-at-a-time
    """
    
    onlyfiles = [f for f in listdir(mainUrl) if isfile(join(mainUrl, f))]
    sortedImgTxt = sorted(onlyfiles)
    # return sortedImgTxt
    # #it = iter(sortedImgTxt)
    combinedList = list()
    # # the only things that should be in the folder should be jpg and txt
    for i, v in  zip(sortedImgTxt[0::2], sortedImgTxt[1::2]):
        print(i, v)
        combinedList.append(list([i,v]))
    
        #check for mismatch of files
        if (i.split('.')[0]) != (v.split('.')[0]):
            print(f'Mismatched file detected {[i,v]} in directory {mainUrl} purging results.\nUser please take action!')
            combinedList = list()
            break
    return combinedList
#             #print (x, next(it))
    # return combinedList
directory = directoryImgTxtload(mainUrl)

136.jpg 136.txt
137.jpg 137.txt
138.jpg 138.txt
139.jpg 139.txt
140.jpg 140.txt
142.jpg 142.txt
143.jpg 143.txt
144.jpg 144.txt
146.jpg 146.txt
147.jpg 147.txt
148.jpg 148.txt
149.jpg 149.txt
150.jpg 150.txt
151.jpg 151.txt
152.jpg 152.txt
153.jpg 153.txt
154.jpg 154.txt
155.jpg 155.txt
156.jpg 156.txt
157.jpg 157.txt
158.jpg 158.txt
159.jpg 159.txt
160.jpg 160.txt
161.jpg 161.txt
162.jpg 162.txt
163.jpg 163.txt
164.jpg 164.txt
165.jpg 165.txt
166.jpg 166.txt
167.jpg 167.txt
168.jpg 168.txt
169.jpg 169.txt
170.jpg 170.txt
171.jpg 171.txt
172.jpg 172.txt
173.jpg 173.txt
174.jpg 174.txt
175.jpg 175.txt
177.jpg 177.txt
178.jpg 178.txt
179.jpg 179.txt
181.jpg 181.txt
182.jpg 182.txt
183.jpg 183.txt
184.jpg 184.txt
185.jpg 185.txt
186.jpg 186.txt
187.jpg 187.txt
188.jpg 188.txt
189.jpg 189.txt
190.jpg 190.txt
191.jpg 191.txt
192.jpg 192.txt
193.jpg 193.txt
194.jpg 194.txt
195.jpg 195.txt
196.jpg 196.txt
197.jpg 197.txt
198.jpg 198.txt
200.jpg 200.txt
202.jpg 202.txt
203.jpg 203.txt
204.jpg 

In [13]:
directory = directoryImgTxtload(mainUrl)

136.jpg 136.txt
137.jpg 137.txt
138.jpg 138.txt
139.jpg 139.txt
140.jpg 140.txt
142.jpg 142.txt
143.jpg 143.txt
144.jpg 144.txt
146.jpg 146.txt
147.jpg 147.txt
148.jpg 148.txt
149.jpg 149.txt
150.jpg 150.txt
151.jpg 151.txt
152.jpg 152.txt
153.jpg 153.txt
154.jpg 154.txt
155.jpg 155.txt
156.jpg 156.txt
157.jpg 157.txt
158.jpg 158.txt
159.jpg 159.txt
160.jpg 160.txt
161.jpg 161.txt
162.jpg 162.txt
163.jpg 163.txt
164.jpg 164.txt
165.jpg 165.txt
166.jpg 166.txt
167.jpg 167.txt
168.jpg 168.txt
169.jpg 169.txt
170.jpg 170.txt
171.jpg 171.txt
172.jpg 172.txt
173.jpg 173.txt
174.jpg 174.txt
175.jpg 175.txt
177.jpg 177.txt
178.jpg 178.txt
179.jpg 179.txt
181.jpg 181.txt
182.jpg 182.txt
183.jpg 183.txt
184.jpg 184.txt
185.jpg 185.txt
186.jpg 186.txt
187.jpg 187.txt
188.jpg 188.txt
189.jpg 189.txt
190.jpg 190.txt
191.jpg 191.txt
192.jpg 192.txt
193.jpg 193.txt
194.jpg 194.txt
195.jpg 195.txt
196.jpg 196.txt
197.jpg 197.txt
198.jpg 198.txt
200.jpg 200.txt
202.jpg 202.txt
203.jpg 203.txt
204.jpg 

In [14]:
def showImgAug(test):
    for i in range(len(test)):
        showImg(test[i][0])

In [15]:
# #!pip install imgaug
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

# imageArray = loadImgToArray(mainUrl+'2.jpg', info=True)
# Bdata = loadingBBox(mainUrl+'2.txt',imageArray,info=True)

# bbs = list()
# for box in Bdata:
#     Bclass,x1,y1,x2,y2 = box
#     bbs.append(BoundingBox(label=Bclass,x1=x1, y1=y1, x2=x2, y2=y2))
# bbs = BoundingBoxesOnImage(bbs,shape=imageArray.shape)

In [16]:
%pylab inline
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg

Populating the interactive namespace from numpy and matplotlib


In [17]:
#setting the pipeline
seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5)), # change brightness, doesn't affect BBs
    iaa.Affine(
        translate_px={"x": 40, "y": 60},
        scale=(0.5, 0.7)
    ) # translate by 40/60px on x/y axis, and scale to 50-70%, affects BBs
])

In [18]:
# Augment BBs and images.
#image_aug, bbs_aug = seq(image=imageArray, bounding_boxes=bbs)

In [19]:
# before picture 
#showImg(imageArray)

In [20]:
# after picture
#showImg(image_aug)

In [21]:
# testUrl = '/home/henri/Desktop/roboflow/quickTest/'
# saveArrayToImg(testUrl+'2.jpg', image_aug,info=False)
# saveBBox(testUrl+'2.txt', bboxNPPacker(bbs_aug), image_aug, info=False)

## End of test

In [22]:
mainUrl = '/home/henri/Desktop/roboflow/'
testUrl = '/home/henri/Desktop/roboflow/quickTest/'

In [23]:
#setting the pipeline
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential([

# the most important part of the image transformation

    sometimes(iaa.CropAndPad(
            percent=(-0.05, 0.2),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        )),
    sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-45, 45), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        ))

# everything bellow is nice but not required 


        # iaa.SomeOf((0, 5),
        #     [
        #         #sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
        #         iaa.OneOf([
        #             iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
        #             iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
        #             iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
        #         ]),
        #         iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
        #         iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
        #         # search either for all edges or for directed edges,
        #         # blend the result with the original image using a blobby mask
        #         iaa.SimplexNoiseAlpha(iaa.OneOf([
        #             iaa.EdgeDetect(alpha=(0.5, 1.0)),
        #             iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
        #         ])),
        #         iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
        #         iaa.OneOf([
        #             iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
        #             iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
        #         ]),
        #         iaa.Invert(0.05, per_channel=True), # invert color channels
        #         iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
        #         iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
        #         # either change the brightness of the whole image (sometimes
        #         # per channel) or change the brightness of subareas
        #         iaa.OneOf([
        #             iaa.Multiply((0.5, 1.5), per_channel=0.5),
        #             iaa.FrequencyNoiseAlpha(
        #                 exponent=(-4, 0),
        #                 first=iaa.Multiply((0.5, 1.5), per_channel=True),
        #                 second=iaa.LinearContrast((0.5, 2.0))
        #             )
        #         ]),
        #         iaa.LinearContrast((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
        #         iaa.Grayscale(alpha=(0.0, 1.0)),
        #         sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
        #         sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
        #         sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
        #     ],
        #     random_order=True)
])

In [24]:
# image_aug, bbs_aug = seq(image=imageArray, bounding_boxes=bbs)
# showImg(imageArray)
# showImg(image_aug)
# bbs_aug

In [25]:
# saveArrayToImg(testUrl+'2.jpg', image_aug,info=False)
# saveBBox(testUrl+'2.txt', bboxNPPacker(bbs_aug), image_aug, info=False)

In [26]:
#easier to load files into dict dict(idx) = [img,box]


In [27]:
# test = augmentedCopiesSingleImage(imageArray,bbs,3)
# for i in range(len(test)):
#     showImg(test[i][0])

In [28]:
mainUrl = '/home/henri/Desktop/roboflow/'
testUrl = '/home/henri/Desktop/roboflow/quickTest/'

## Loading files and packaging them (test)

In [29]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mainUrl) if isfile(join(mainUrl, f))]
sortedImgTxt = sorted(onlyfiles)

In [30]:
# # https://stackoverflow.com/questions/21752610/iterate-every-2-elements-from-list-at-a-time
# it = iter(sortedImgTxt)
# combinedList = list()
# # the only things that should be in the folder should be jpg and txt
# for x in it:
#     combinedList.append([x, next(it)])
#     # check for mismatch of files
#     if (combinedList[0][0].split('.')[0]) != (combinedList[0][1].split('.')[0]):
#         print(f'Mismatched file detected {combinedList[0]} in directory {mainUrl} purging results.\nUser please take action!')
#         combinedList = list()
#         break
#     #print (x, next(it))
# #combinedList

## Loading files and packaging them (ready)

In [31]:
testli = ['100.jpg', '100.txt']

In [32]:
fileNameAugGen(69,testli)

['100aug69.jpg', '100aug69.txt']

In [33]:
# directory = directoryImgTxtload(mainUrl)
# imageArray = loadImgToArray(mainUrl + directory[1][0], info=False)
# Bdata = loadingBBox(mainUrl + directory[1][1],imageArray,info=False)

# bbs = list()
# for box in Bdata:
#     Bclass,x1,y1,x2,y2 = box
#     bbs.append(BoundingBox(label=Bclass,x1=x1, y1=y1, x2=x2, y2=y2))
# bbs = BoundingBoxesOnImage(bbs,shape=imageArray.shape)

# augmentedImagesList = augmentedCopiesSingleImage(imageArray,bbs,np.random.randint(4,high=9))


In [34]:
# showImgAug(augmentedImagesList)

In [35]:
# for i in range(len(augmentedImagesList)):
#     jpgNewName, txtNewName = fileNameAugGen(i,directory[1])
#     saveArrayToImg(testUrl + jpgNewName, augmentedImagesList[i][0],info=False)
#     #! error bellow
#     #url,
#     saveBBox(testUrl + txtNewName, bboxNPPacker(augmentedImagesList[i][1]), augmentedImagesList[i][0], info=False)


In [36]:
#directory = directoryImgTxtload(mainUrl)
#directory

In [37]:
# now everything together
mainUrl = '/home/henri/Desktop/roboflow/'
testUrl = '/home/henri/Desktop/roboflow/224.ext/'
from tqdm import  tqdm
directory = directoryImgTxtload(mainUrl)
for f in tqdm(range(len(directory))):
    
    imageArray = loadImgToArray(mainUrl + directory[f][0], info=False)
    Bdata = loadingBBox(mainUrl + directory[f][1],imageArray,info=False)
    print(f'\nworking on {directory[f][0]} {directory[f][1]}')
    bbs = list()
    for box in Bdata:
        Bclass,x1,y1,x2,y2 = box
        bbs.append(BoundingBox(label=Bclass,x1=x1, y1=y1, x2=x2, y2=y2))
    bbs = BoundingBoxesOnImage(bbs,shape=imageArray.shape)
    # I am running out of memory maybe this will help
    augmentedImagesList = augmentedCopiesSingleImage(imageArray,bbs,np.random.randint(3,high=7))
    for i in range(len(augmentedImagesList)):
        jpgNewName, txtNewName = fileNameAugGen(i,directory[f])
        saveArrayToImg(testUrl + jpgNewName, augmentedImagesList[i][0],info=False)
        #! error bellow
        #url,
        saveBBox(testUrl + txtNewName, bboxNPPacker(augmentedImagesList[i][1]), augmentedImagesList[i][0], info=False)


  0%|          | 0/66 [00:00<?, ?it/s]136.jpg 136.txt
137.jpg 137.txt
138.jpg 138.txt
139.jpg 139.txt
140.jpg 140.txt
142.jpg 142.txt
143.jpg 143.txt
144.jpg 144.txt
146.jpg 146.txt
147.jpg 147.txt
148.jpg 148.txt
149.jpg 149.txt
150.jpg 150.txt
151.jpg 151.txt
152.jpg 152.txt
153.jpg 153.txt
154.jpg 154.txt
155.jpg 155.txt
156.jpg 156.txt
157.jpg 157.txt
158.jpg 158.txt
159.jpg 159.txt
160.jpg 160.txt
161.jpg 161.txt
162.jpg 162.txt
163.jpg 163.txt
164.jpg 164.txt
165.jpg 165.txt
166.jpg 166.txt
167.jpg 167.txt
168.jpg 168.txt
169.jpg 169.txt
170.jpg 170.txt
171.jpg 171.txt
172.jpg 172.txt
173.jpg 173.txt
174.jpg 174.txt
175.jpg 175.txt
177.jpg 177.txt
178.jpg 178.txt
179.jpg 179.txt
181.jpg 181.txt
182.jpg 182.txt
183.jpg 183.txt
184.jpg 184.txt
185.jpg 185.txt
186.jpg 186.txt
187.jpg 187.txt
188.jpg 188.txt
189.jpg 189.txt
190.jpg 190.txt
191.jpg 191.txt
192.jpg 192.txt
193.jpg 193.txt
194.jpg 194.txt
195.jpg 195.txt
196.jpg 196.txt
197.jpg 197.txt
198.jpg 198.txt
200.jpg 200.txt
20